In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import json
import cv2
import sklearn
from tensorflow.keras import regularizers
from tensorflow.keras import layers
import os

In [2]:
GESTURE_TYPES = 11
LABEL_DICT = {k:i for i,k in enumerate([21, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33])}
CONNECTION_LABELS = [
    (0, 1), (1, 2), (2, 3), (3, 4),
    (5, 6), (6, 7), (7, 8),
    (9, 10), (10, 11), (11, 12),
    (13, 14), (14, 15), (15, 16),
    (17, 18), (18, 19), (19, 20),
    (0, 5), (5, 9), (9, 13), (13, 17), (0, 17)
]

In [3]:
def load_keypoint_sequences(data_path='gesture_recognition/Fall 2020/data', sequence_length=5):
    keypoints = []
    labels = []
    for subjectName in os.listdir(data_path):
        if not (subjectName.startswith("Subject") or subjectName.startswith("subject")): continue
        # subjectNum = int(re.findall(r'(\d+)', subjectName)[0])
        for sceneName in os.listdir(os.path.join(data_path, subjectName)):
            if not (sceneName.startswith("Scene") or subjectName.startswith("scene")): continue
            for groupEntry in os.scandir(os.path.join(data_path, subjectName, sceneName)):
                with open(groupEntry, 'r') as f:
                    groupData = json.load(f)
                    for gesture in groupData:
                        # print(gesture['label'], gesture['keypoints'])
                        for i in range(len(gesture['keypoints'])):
                            if not gesture['keypoints'][i]:
                                gesture['keypoints'][i] = [[np.nan, np.nan, np.nan] for _ in range(21)]
                        for start_frame in range(len(gesture['keypoints']) - sequence_length + 1):
                            keypoints.append(gesture['keypoints'][start_frame: start_frame + sequence_length])
                            labels.append(LABEL_DICT[gesture['label']])
    keypoints = np.array(keypoints)
    labels = np.array(labels)
    return keypoints, labels
keypoints, labels = load_keypoint_sequences()
print(keypoints.shape, labels.shape)

(23346, 5, 21, 3) (23346,)


In [4]:
def generate_connection_angles_from_sequences(keypoints, keypoints_num=21, keypoints_dimensions=3):
    connections = []
    for connection in CONNECTION_LABELS:
        connections.append(keypoints[..., connection[1], :] - keypoints[..., connection[0], :])
    connections = np.stack(connections, axis = -2)
    tensor1 = connections[..., np.newaxis].repeat(keypoints_num, -1).transpose(0,1,2,4,3)
    tensor2 = connections[..., np.newaxis].repeat(keypoints_num, -1).transpose(0,1,4,2,3)
    angles = (tensor1*tensor2).sum(axis=-1)/np.linalg.norm(tensor1,axis=-1)/np.linalg.norm(tensor2,axis=-1)
    angles = angles.transpose(2,3,0,1)[np.triu_indices(21, k = 1)].transpose(1,2,0)
    return np.arccos(angles)
angles = generate_connection_angles_from_sequences(keypoints)
print(angles.shape)

(23346, 5, 210)


In [5]:
def generate_joint_distances_from_sequences(keypoints, keypoints_num=21, keypoints_dimensions=3):
    connections = []
    for connection in CONNECTION_LABELS:
        connections.append(keypoints[..., connection[1], :] - keypoints[..., connection[0], :])
    connections = np.stack(connections, axis = -2)
    tensor1 = connections[..., np.newaxis].repeat(keypoints_num, -1).transpose(0,1,2,4,3)
    tensor2 = connections[..., np.newaxis].repeat(keypoints_num, -1).transpose(0,1,4,2,3)
    distances = np.linalg.norm(tensor1-tensor2,axis=-1)
    return distances
distances = generate_joint_distances_from_sequences(keypoints)
print(distances.shape)

(23346, 5, 21, 21)


In [6]:
def visualize_keypoint_sequences(keypoints):
    for sequence in keypoints:
        for points in sequence:
            img = np.zeros((480, 640, 3))
            for point in points:
                x, y, z = point
                if np.isnan(x):
                    continue
                cv2.circle(img, (int(x), int(y)), 4, (255, 0, 0), 2)
            for connection in CONNECTION_LABELS:
                if np.isnan(points[connection[0]][0]):
                    continue
                x0, y0, z0 = points[connection[0]]
                x1, y1, z1 = points[connection[1]]
                cv2.line(img, (int(x0), int(y0)), (int(x1), int(y1)), (0, 255, 0), 2)
            cv2.imshow("Key Points", img)
            key = cv2.waitKey(1)
            if key == 27:
                cv2.destroyAllWindows()
                cv2.waitKey(1) # cannot close window on macOS without this line
                return
# visualize_keypoint_sequences(keypoints)

In [7]:
def process_sequence_features(keypoints, angles):
    data_length = keypoints.shape[0]
    sequence_length = keypoints.shape[1]
    keypoints = keypoints.reshape(data_length*sequence_length, -1)
    angles = angles.reshape(data_length*sequence_length, -1)
    features = np.concatenate((keypoints, angles), -1)
    df = pd.DataFrame(features)
    df = (df-df.mean())/df.std()
    df = df.fillna(0)
    features = df.to_numpy().reshape(data_length, sequence_length, -1)
    return features
X = process_sequence_features(keypoints, angles)
#normalizer = tf.keras.layers.experimental.preprocessing.Normalization()
#normalizer.adapt(X)
# X_train, X_val, y_train, y_val = train_test_split(processed_keypoints, labels, test_size=0.2, random_state=0)
print(X.shape)

(23346, 5, 273)


## RNN Classification

In [8]:
model_lstm = tf.keras.Sequential([layers.Masking(), layers.LSTM(GESTURE_TYPES, activation=None), layers.Activation('softmax')])
model_lstm.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_lstm.fit(X, labels, epochs=20, validation_split=0.2)

Train on 18676 samples, validate on 4670 samples
Epoch 1/20
18676/18676 [==============================] - 17s 920us/sample - loss: 0.7352 - accuracy: 0.7601 - val_loss: 0.6725 - val_accuracy: 0.7884
Epoch 2/20
18676/18676 [==============================] - 12s 663us/sample - loss: 0.4159 - accuracy: 0.8669 - val_loss: 0.6619 - val_accuracy: 0.8024
Epoch 3/20
18676/18676 [==============================] - 10s 512us/sample - loss: 0.3599 - accuracy: 0.8847 - val_loss: 0.6740 - val_accuracy: 0.8118
Epoch 4/20
18676/18676 [==============================] - 12s 622us/sample - loss: 0.3272 - accuracy: 0.8958 - val_loss: 0.6572 - val_accuracy: 0.8146
Epoch 5/20
18676/18676 [==============================] - 13s 677us/sample - loss: 0.2934 - accuracy: 0.9059 - val_loss: 0.6391 - val_accuracy: 0.8218
Epoch 6/20
18676/18676 [==============================] - 12s 639us/sample - loss: 0.2777 - accuracy: 0.9105 - val_loss: 0.6755 - val_accuracy: 0.8161
Epoch 7/20
18676/18676 [=====================

In [9]:
model_bilstm = tf.keras.Sequential([layers.Masking(), layers.Bidirectional(layers.LSTM(GESTURE_TYPES, activation=None)), layers.Activation('softmax')])
model_bilstm.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_bilstm.fit(X, labels, epochs=20, validation_split=0.2)

Train on 18676 samples, validate on 4670 samples
Epoch 1/20
18676/18676 [==============================] - 31s 2ms/sample - loss: 0.7706 - accuracy: 0.7711 - val_loss: 0.7010 - val_accuracy: 0.7906
Epoch 2/20
18676/18676 [==============================] - 16s 881us/sample - loss: 0.4198 - accuracy: 0.8658 - val_loss: 0.6489 - val_accuracy: 0.8051
Epoch 3/20
18676/18676 [==============================] - 20s 1ms/sample - loss: 0.3717 - accuracy: 0.8809 - val_loss: 0.6710 - val_accuracy: 0.8019
Epoch 4/20
18676/18676 [==============================] - 21s 1ms/sample - loss: 0.3305 - accuracy: 0.8931 - val_loss: 0.6538 - val_accuracy: 0.8062
Epoch 5/20
18676/18676 [==============================] - 21s 1ms/sample - loss: 0.3045 - accuracy: 0.9004 - val_loss: 0.6926 - val_accuracy: 0.8069
Epoch 6/20
18676/18676 [==============================] - 20s 1ms/sample - loss: 0.2811 - accuracy: 0.9088 - val_loss: 0.6681 - val_accuracy: 0.8131
Epoch 7/20
18676/18676 [==============================]

Making LSTM bidirectional does not improve the accuracy by a lot.

In [10]:
model_rnn = tf.keras.Sequential([layers.Masking(), layers.SimpleRNN(GESTURE_TYPES, activation=None), layers.Activation('softmax')])
model_rnn.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_rnn.fit(X, labels, epochs=20, validation_split=0.2)

Train on 18676 samples, validate on 4670 samples
Epoch 1/20
18676/18676 [==============================] - 10s 509us/sample - loss: 1.1371 - accuracy: 0.6599 - val_loss: 0.8616 - val_accuracy: 0.7343
Epoch 2/20
18676/18676 [==============================] - 6s 305us/sample - loss: 0.6330 - accuracy: 0.8057 - val_loss: 0.7836 - val_accuracy: 0.7608
Epoch 3/20
18676/18676 [==============================] - 6s 331us/sample - loss: 0.5504 - accuracy: 0.8266 - val_loss: 0.7540 - val_accuracy: 0.7681
Epoch 4/20
18676/18676 [==============================] - 6s 317us/sample - loss: 0.5152 - accuracy: 0.8398 - val_loss: 0.7375 - val_accuracy: 0.7715
Epoch 5/20
18676/18676 [==============================] - 6s 332us/sample - loss: 0.4830 - accuracy: 0.8483 - val_loss: 0.6947 - val_accuracy: 0.7850
Epoch 6/20
18676/18676 [==============================] - 6s 333us/sample - loss: 0.4708 - accuracy: 0.8520 - val_loss: 0.7107 - val_accuracy: 0.7809
Epoch 7/20
18676/18676 [==========================

Simple RNN gives a slightly worse performance.

In [11]:
model_lstm4 = tf.keras.Sequential([layers.Masking(), layers.LSTM(128), layers.Dense(GESTURE_TYPES), layers.Activation('softmax')])
model_lstm4.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_lstm4.fit(X, labels, epochs=20, validation_split=0.2)

Train on 18676 samples, validate on 4670 samples
Epoch 1/20
18676/18676 [==============================] - 26s 1ms/sample - loss: 0.4525 - accuracy: 0.8583 - val_loss: 0.5977 - val_accuracy: 0.8069
Epoch 2/20
18676/18676 [==============================] - 19s 1ms/sample - loss: 0.2183 - accuracy: 0.9324 - val_loss: 0.5826 - val_accuracy: 0.8169
Epoch 3/20
18676/18676 [==============================] - 19s 1ms/sample - loss: 0.1347 - accuracy: 0.9620 - val_loss: 0.6071 - val_accuracy: 0.8109
Epoch 4/20
18676/18676 [==============================] - 19s 1ms/sample - loss: 0.0889 - accuracy: 0.9767 - val_loss: 0.6447 - val_accuracy: 0.8233
Epoch 5/20
18676/18676 [==============================] - 19s 1ms/sample - loss: 0.0615 - accuracy: 0.9847 - val_loss: 0.6727 - val_accuracy: 0.8208
Epoch 6/20
18676/18676 [==============================] - 19s 1ms/sample - loss: 0.0501 - accuracy: 0.9870 - val_loss: 0.6939 - val_accuracy: 0.8169
Epoch 7/20
18676/18676 [==============================] -

Increasing depth may improve performance.

## MLP on Concatenated Sequence of Data

MLP generally has a worse performance than RNN.

In [12]:
X_expanded = X.reshape((X.shape[0],-1))
model_mlp = tf.keras.Sequential([layers.Dense(GESTURE_TYPES), layers.Activation('softmax')])
model_mlp.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_mlp.fit(X_expanded, labels, epochs=20, validation_split=0.2)

Train on 18676 samples, validate on 4670 samples
Epoch 1/20
18676/18676 [==============================] - 4s 206us/sample - loss: 0.6502 - accuracy: 0.8016 - val_loss: 0.7110 - val_accuracy: 0.7908
Epoch 2/20
18676/18676 [==============================] - 2s 130us/sample - loss: 0.4873 - accuracy: 0.8508 - val_loss: 0.6974 - val_accuracy: 0.7953
Epoch 3/20
18676/18676 [==============================] - 2s 133us/sample - loss: 0.4520 - accuracy: 0.8600 - val_loss: 0.7138 - val_accuracy: 0.7972
Epoch 4/20
18676/18676 [==============================] - 3s 137us/sample - loss: 0.4281 - accuracy: 0.8677 - val_loss: 0.6949 - val_accuracy: 0.7979
Epoch 5/20
18676/18676 [==============================] - 3s 154us/sample - loss: 0.4088 - accuracy: 0.8711 - val_loss: 0.6887 - val_accuracy: 0.7985
Epoch 6/20
18676/18676 [==============================] - 3s 150us/sample - loss: 0.3994 - accuracy: 0.8740 - val_loss: 0.7386 - val_accuracy: 0.7946
Epoch 7/20
18676/18676 [===========================

In [13]:
model_mlp2 = tf.keras.Sequential([layers.Dense(256), layers.Dense(GESTURE_TYPES), layers.Activation('softmax')])
model_mlp2.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_mlp2.fit(X_expanded, labels, epochs=20, validation_split=0.2)

Train on 18676 samples, validate on 4670 samples
Epoch 1/20
18676/18676 [==============================] - 5s 246us/sample - loss: 0.9507 - accuracy: 0.7939 - val_loss: 0.8666 - val_accuracy: 0.7745
Epoch 2/20
18676/18676 [==============================] - 4s 198us/sample - loss: 0.5066 - accuracy: 0.8494 - val_loss: 0.7545 - val_accuracy: 0.7835
Epoch 3/20
18676/18676 [==============================] - 4s 193us/sample - loss: 0.4602 - accuracy: 0.8577 - val_loss: 0.6823 - val_accuracy: 0.7951
Epoch 4/20
18676/18676 [==============================] - 4s 193us/sample - loss: 0.4511 - accuracy: 0.8606 - val_loss: 0.7114 - val_accuracy: 0.7865
Epoch 5/20
18676/18676 [==============================] - 4s 194us/sample - loss: 0.4367 - accuracy: 0.8615 - val_loss: 0.7116 - val_accuracy: 0.7955
Epoch 6/20
18676/18676 [==============================] - 4s 204us/sample - loss: 0.4251 - accuracy: 0.8670 - val_loss: 0.7517 - val_accuracy: 0.7767
Epoch 7/20
18676/18676 [===========================